In [43]:
%%time
import os
import threading
import subprocess
import numpy as np
from random import randrange
import math

CPU times: user 0 ns, sys: 17 µs, total: 17 µs
Wall time: 21.9 µs


In [44]:
source_dir = "../data/source/"
source_files = []
for j in sorted(os.listdir(source_dir)):
    source_files += [f"{source_dir}{j}"]
source_files[-1]

'../data/source/922_5400.mkv'

In [45]:
# splitting into file_time (in seconds) +- delta_seconds
file_time = 5.0 # seconds
delta = 0.5 # seconds
delta_step = 0.01 # seconds
min_sr = 8000
max_sr = 48000
resulting_durations = np.around(np.arange(file_time - delta, file_time + delta, delta_step), decimals = 2)
sampling_rates = np.arange(min_sr, max_sr, 1)

# duration in seconds, nned to pass file.mkv as last argument
check_file_duration_command = "ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1"

# now generate list of ffmpeg commands:
start_delta = 606
ffmpeg_commands = []
for file_index, file in enumerate(source_files):
    file_duration = int(float(subprocess.getoutput(f"{check_file_duration_command} {file}")))
    start_times = np.around(np.arange(0, file_duration, math.ceil(file_time + delta)), decimals = 1)
    start_times = start_times[1:-1] # let's remove first and last elements to be safe
    for start_index, start in enumerate(start_times):
        selected_duration = resulting_durations[randrange(len(resulting_durations))]
        selected_sample_rate = sampling_rates[randrange(len(sampling_rates))]
        output_file = f"../data/sliced/{file_index + start_delta}_{int(start_index*math.ceil(file_time + delta))}_{selected_duration}_{selected_sample_rate}.wav"
        ffmpeg_commands += [f"\
        ffmpeg -y -i {file} \
            -ss {start} \
            -t {selected_duration} \
            -ab 320k \
            -ar {selected_sample_rate} \
            -ac 1 \
            -vn {output_file} \
            && chown 1000:1000 {output_file} \
            "]

In [46]:
" ".join(ffmpeg_commands[0].split())

'ffmpeg -y -i ../data/source/606_0.mkv -ss 6 -t 5.26 -ab 320k -ar 37277 -ac 1 -vn ../data/sliced/606_0_5.26_37277.wav && chown 1000:1000 ../data/sliced/606_0_5.26_37277.wav'

In [47]:
len(ffmpeg_commands)

29595

In [48]:
%%time
### SET THREADS NUMBER 
nproc = 6
bash_command = "sleep 5" # for tests

semaphore = threading.Semaphore(nproc)
def run_command(cmd):
    with semaphore:
        os.system(cmd)
threads = []

for task in sorted(ffmpeg_commands):
    threads += [threading.Thread(target=run_command, args=(task,))]

for j in threads:
    j.start()
for j in threads:
    j.join()

CPU times: user 8.01 s, sys: 31min 59s, total: 32min 7s
Wall time: 1h 39min 36s
